# Logging within AzureML

## Using the AzureML SDK log_* methods

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="logging-experiment")
run = exp.start_logging()

In [ ]:
print(run.get_details())
print(run.get_portal_url())

In [ ]:
run.log("nrmse", 0.01)
run.log(name="nrmse", value=0.015)

In [ ]:
run.log_list("accuracies", [0.5, 0.57, 0.62])

In [ ]:
run.log_table("table", {"x": [1, 2], "y": [0.1, 0.2]})
run.log_row("table", x=3, y=0.3)

In [ ]:
# Create child runs to isolate runs
with run.child_run() as child_run:
    child_run.log("child_metric", 0.01)
    child_run.parent.log("metric_from_child", 0.02)

# Previous child run is completed by now (got outside the with scope above)
# If I want to cancel a child run I can do the following
child_run = run.child_run()
child_run.log("canceled_child_metric", 0.03)
child_run.cancel()  # I cancelled the child run

In [ ]:
run.complete() # This was automatically called by the with scope above
print(run.get_status())

## MLflow integration

In [ ]:
# You will need azureml-mlflow pip package
# which already exists in your compute instance
!pip show azureml-mlflow

In [ ]:
import mlflow

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment("mlflow-experiment")

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLars
from sklearn import datasets

with mlflow.start_run():
    mlflow.sklearn.autolog()
    # Normal sklearn training code
    alpha_param = 0.9
    diabetes = datasets.load_diabetes()
    X = diabetes.data
    y = diabetes.target
    pipe = make_pipeline(StandardScaler(), LassoLars(alpha=alpha_param))
    pipe.fit(X, y)
    # Track using the MLflow API
    mlflow.log_metric("custom_metric", 0.9)
    # MLflow already tracks lassolars__alpha
    mlflow.log_param("alpha", alpha_param)